In [ ]:
import pathlib

# Setup

## Parse trajectory

In [ ]:
from analyzer.io.trajectory_parser import parse_trajectory

trajectory = parse_trajectory(pathlib.Path("demos/crossing/CROSSING_90_D_6.txt"), 16.)

## Define geometry

In [ ]:
from analyzer.data.geometry import Geometry

### Define geometry in code

In [ ]:
import pygeos
geometry = Geometry(pygeos.polygons([(-9, 3), (-2, 3), (-2, 10), (2, 10), (2, 3), (9, 3), (9, -1), (2, -1), (2, -8), (-2, -8), (-2, -1), (-9, -1), (-9, 3)]))

### Define geometry in file

The same format as for JuPedSim geometries can be used as well directly from files.

**Note:** Currently obstacles will not be parsed from the file!

In [ ]:
from analyzer.io.geometry_parser import parse_geometry

geometry = parse_geometry('demos/crossing/geometry.xml')

## Define measurement areas and lines

In [ ]:
import pygeos

measurement_area = pygeos.polygons([(-2, -1), (-2, 3), (2, 3), (2, -1)])

measurement_line_left = pygeos.linestrings([(-3, 3), (-3, -1)])
measurement_line_right = pygeos.linestrings([(3, 3), (3, -1)])

measurement_line_top = pygeos.linestrings([(-2, 4), (2, 4)])
measurement_line_bottom = pygeos.linestrings([(-2, -2), (2, -2)])

## Plot trajectories, geometry, and measurement areas/lines

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15, 20))
ax1 = fig.add_subplot(111, aspect="equal")

# Plot geometry
ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color='k')

## Plot measurement area
ax1.plot(*pygeos.to_shapely(measurement_area).exterior.xy, color='g')
ax1.fill(*pygeos.to_shapely(measurement_area).exterior.xy, color='g', alpha=0.3)

## Plot measurement lines
ax1.plot(*pygeos.to_shapely(measurement_line_left).xy, color='b')
ax1.plot(*pygeos.to_shapely(measurement_line_right).xy, color='b')
ax1.plot(*pygeos.to_shapely(measurement_line_top).xy, color='orange')
ax1.plot(*pygeos.to_shapely(measurement_line_bottom).xy, color='orange')

# Plot trajectories
for id, ped in trajectory.data.groupby("ID"):
    p = ax1.plot(ped["X"], ped["Y"], label=id, alpha=0.1, color='r')
    ax1.scatter(
        ped[ped.frame == ped.frame.max()]["X"],
        ped[ped.frame == ped.frame.max()]["Y"],
        c=p[-1].get_color(),
        marker="x",
    )
plt.show()

# Methods A

## Compute n-t and flow

In [ ]:
from analyzer.methods.flow_calculator import compute_n_t
from analyzer.methods.flow_calculator import compute_flow

from analyzer.methods.velocity_calculator import compute_individual_velocity

individual_speed = compute_individual_velocity(trajectory.data, trajectory.frame_rate, 10)

nt_left, crossing_left = compute_n_t(trajectory.data, measurement_line_left, trajectory.frame_rate)
nt_right, crossing_right = compute_n_t(trajectory.data, measurement_line_right, trajectory.frame_rate)

delta_t = 100
flow_left = compute_flow(nt_left, crossing_left, individual_speed, delta_t, trajectory.frame_rate)
flow_right = compute_flow(nt_right, crossing_right, individual_speed, delta_t, trajectory.frame_rate)

## Plot n-t diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

ax1.plot(nt_left.index, nt_left['Cumulative pedestrians'], label='left')
ax1.plot(nt_right.index, nt_right['Cumulative pedestrians'], label='right')

ax1.legend()
plt.show()

# Method B

## Compute individual velocity and density while passing the measurement area

In [ ]:
from analyzer.methods.velocity_calculator import compute_passing_speed
from analyzer.methods.density_calculator import compute_passing_density, compute_classic_density
from analyzer.methods.method_utils import compute_frame_range_in_area

density_per_frame = compute_classic_density(trajectory.data, measurement_area)
frames_in_area = compute_frame_range_in_area(trajectory.data, measurement_area)
passing_speed = compute_passing_speed(frames_in_area, trajectory.frame_rate)
passing_density = compute_passing_density(density_per_frame, frames_in_area)

## Plot fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

ax1.scatter(passing_density['density'], passing_speed['speed'])
ax1.set_xlim(left=0, right=5)
ax1.set_ylim(bottom=-0.2, top=3)

ax1.set_xlabel("rho / 1/m^2")
ax1.set_ylabel("v/ m/s")
ax1.grid()
plt.show()

# Method C

## Compute density and velocity

In [ ]:
from analyzer.methods.density_calculator import compute_classic_density
from analyzer.methods.velocity_calculator import compute_mean_velocity_per_frame
from analyzer.methods.method_utils import get_peds_in_area

peds_in_area = get_peds_in_area(trajectory.data, measurement_area)
mean_speed_area, individual_speed_area = compute_mean_velocity_per_frame(
    peds_in_area, trajectory.frame_rate, 5
)

classic_density = compute_classic_density(peds_in_area, measurement_area)

## Plot velocity, density and fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))

ax0.set_title("Velocity")
ax0.scatter(mean_speed_area.index, mean_speed_area, alpha=0.5, marker="x")
ax0.set_xlim(
    left=0,
)
ax0.set_ylim(bottom=-0.2, top=3)
ax0.set_xlabel("frame")
ax0.set_ylabel("v / m/s")
ax0.grid()

ax1.set_title("Density")
ax1.scatter(classic_density.index, classic_density['classic density'], alpha=0.5, marker="x")
ax1.set_xlim(left=0)
ax1.set_ylim(bottom=0, top=4)
ax1.set_xlabel("frame")
ax1.set_ylabel("rho / 1/m^2")
ax1.grid()

ax2.set_title("Fundamental Diagram")

ax2.scatter(
    classic_density['classic density'], mean_speed_area,
    alpha=0.2,
    marker="x",
)
ax2.set_aspect("equal")
ax2.set_xlim(left=0, right=5)
ax2.set_ylim(bottom=-0.2, top=3)

ax2.set_xlabel("rho / 1/m^2")
ax2.set_ylabel("v/ m/s")
ax2.grid()
